## Granger Causality Testing

**Functions**

`VARResults.test_causality`

### Exercise 92
Using the data and the models selected in the previous exercise, is there evidence of Granger Causality between the series?

In [1]:
import pandas as pd
import statsmodels.tsa.api as tsa

data = pd.read_hdf("./data/var-data.h5", "var_data")
var_data = data[["spread", "gs1", "deflg"]]
mod = tsa.VAR(var_data)
res = mod.fit(2, ic=None)

In [2]:
gc = {}
for caused in var_data:
    for causing in var_data:
        if caused == causing:
            continue
        test_res = res.test_causality(caused, causing)
        gc[(caused, causing)] = [test_res.test_statistic, test_res.pvalue]

gc = pd.DataFrame(gc).T
gc.columns = ["Test Stat", "PValue"]
gc.index = gc.index.set_names(["caused", "causing"])
gc

Test Stat    PValue
caused causing                     
spread gs1       2.182779  0.113424
       deflg     2.446992  0.087225
gs1    spread    0.817780  0.441793
       deflg     6.826381  0.001152
deflg  spread    0.628702  0.533557
       gs1       1.488053  0.226460

#### Explanation

Using a VAR(2), we see that deflator growth Granger causes GS1, indicating
that the restriction on the coefficients is rejected. Deflator growth
also appears to GC spread but only if using a 10% test. Since the Term
spread is never caused or causes (at least at the 5% level), it could
be plausible excluded from this model.

### Exercise 93
What if the 10 year and the 1 year are both used, but the spread is omitted?

In [3]:
var_data = data[["gs10", "gs1", "deflg"]]
mod = tsa.VAR(var_data)
res = mod.fit(2, ic=None)

gc = {}
for caused in var_data:
    for causing in var_data:
        if caused == causing:
            continue
        test_res = res.test_causality(caused, causing)
        gc[(caused, causing)] = [test_res.test_statistic, test_res.pvalue]

gc = pd.DataFrame(gc).T
gc.columns = ["Test Stat", "PValue"]
gc.index = gc.index.set_names(["caused", "causing"])
gc

Test Stat    PValue
caused causing                     
gs10   gs1       0.240809  0.786051
       deflg     6.651588  0.001367
gs1    gs10      0.817780  0.441793
       deflg     6.826381  0.001152
deflg  gs10      0.628702  0.533557
       gs1       4.501112  0.011390

#### Explanation

Using a model that is equivalent to the first specification but with a
different parameterization produces different results. This occurs since
we are changing both the dependent and the regressor.